### <span style="font-size:22px;">Synthea Health: Data Analysis and Insights Generation</span>


### 1. Initialisation

In [ ]:
!pip install -r ./requirements.txt

### 2. Data Wrangling & Cleaning

#### 1. Import the required functions for reading CSV files, Cleaning data and transformations

In [ ]:
from utils.utils import read_data,clean_patient_data,transform_patient_data,clean_transform_condition_data,delete_from_data,clean_transform_procedure_data,clean_transform_medication_data,clean_transform_encounters_data
import json

In [ ]:
# Read the file path from Config file to retrive the Input folder where all Source files are maintained
#config_file_path = r'C:\Users\veena.vemula\Documents\GitHub\SDE-Skills-Test\Config\config.json'
config_file_path = './Config/config.json'

# Load the JSON file
with open(config_file_path, 'r') as file:
    config = json.load(file)

# Access the value of 'input_folder'
input_folder = config.get('input_folder')
print("Input Folder:", input_folder)

In [85]:
# Call the read function to load patients data
df_patients = read_data(input_folder,"patients")
df_conditions = read_data(input_folder,"conditions")
df_encounters = read_data(input_folder,"encounters")
df_medications = read_data(input_folder,"medications")
df_procedures = read_data(input_folder,"procedures")
df_organizations = read_data(input_folder,"organizations")

# Check the result (show the first few rows)
if df_patients is not None:
    print(df_patients.head())
else:
    print("Failed to load the Patients CSV file.")

# Check the result (show the first few rows)
if df_conditions is not None:
    print(df_conditions.head())
else:
    print("Failed to load the Conditions CSV file.")

# Check the result (show the first few rows)
if df_encounters is not None:
    print(df_encounters.head())
else:
    print("Failed to load the Encounters CSV file.")

# Check the result (show the first few rows)
if df_medications is not None:
    print(df_medications.head())
else:
    print("Failed to load the medications CSV file.")

# Check the result (show the first few rows)
if df_procedures is not None:
    print(df_procedures.head())
else:
    print("Failed to load the procedures CSV file.")

# Check the result (show the first few rows)
if df_organizations is not None:
    print(df_organizations.head())
else:
    print("Failed to load the organizations CSV file.")



                                     Id   BIRTHDATE DEATHDATE          SSN  \
0  b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85  2019-02-17       NaN  999-65-3251   
1  c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8  2005-07-04       NaN  999-49-3323   
2  339144f8-50e1-633e-a013-f361391c4cff  1998-05-11       NaN  999-10-8743   
3  d488232e-bf14-4bed-08c0-a82f34b6a197  2003-01-28       NaN  999-56-6057   
4  217f95a3-4e10-bd5d-fb67-0cfb5e8ba075  1993-12-23       NaN  999-91-4320   

     DRIVERS    PASSPORT PREFIX       FIRST            LAST SUFFIX  ...  \
0        NaN         NaN    NaN    Damon455      Langosh790    NaN  ...   
1  S99941126         NaN    NaN       Thi53       Wunsch504    NaN  ...   
2  S99996708  X75063318X    Mr.      Chi716  Greenfelder433    NaN  ...   
3  S99929424         NaN    Ms.  Phillis443       Walter473    NaN  ...   
4  S99991143  X44132498X    Mr.  Jerrold404       Herzog843    NaN  ...   

                         BIRTHPLACE                           ADDRESS  \
0  Midd

In [ ]:
# List of DataFrames
dataframes = {
    "Patients": df_patients,
    "conditions": df_conditions ,
    "encounters": df_encounters,
    "Medications":df_medications,
    "Procedures": df_procedures,
    "organizations" :df_organizations
}

# Loop through all DataFrames and check for duplicates
for name, df in dataframes.items():
    num_duplicates = df.duplicated().sum()
    
    if num_duplicates > 0:
        print(f"Number of duplicate rows in {name} file: {num_duplicates}")
    else:
        print(f"No duplicate values found in {name} file")


#### Cleaning and transforming the Patients file

In [ ]:
""" Use Value_counts() to inspect the data and identify if any paterns """

df_patients['ADDRESS'].value_counts()
df_patients['FIRST'].value_counts()


Inspecting the Value counts on each of columns dint revela any significant data issues or invalid categorical columns and inconsistent data

In [ ]:
missing_values = df_patients .isnull().sum()
print("Missing values in the DataFrame:")
print(missing_values)

By inspecting the Missing values in patients file
1) Deat code can be blank if the patient is alive
2) Drivers,Passport and Prefix , Suffic Maiden column have null's but at this stage these columns are not significant 

In [ ]:
## Clean patient data  
df_patients = clean_patient_data(df_patients)
df_patients.head()

Check for any duplicate column values
Rename the key value to Patientid - This has to be mainted same across all dataframes
Retrived the required columns and renamed them to make more sense.

In [ ]:
print("\nData types of each column:")
df_patients.dtypes
## all the columns are of object type. we can cast it to desired type for e.g. convert BIRTHDATE to date
## GENDER, RACE, ETHNICITY ... to string

In [ ]:
df_patients = transform_patient_data(df_patients)
df_patients.dtypes

Transformations have been applied step by step as below.
1) Tranform Birthdatetime to required format to calculate patients age.
2) Type casting to convert object types to strings i.e gender, Race and Ethinicity
3) Describe age bins and label

In [ ]:
df_patients.to_csv("./output/patients_cleaned.csv",index=False)

Export the cleaned file to output folder

###Cleaning and Transforming Conditions file

In [ ]:
missing_values = df_conditions.isnull().sum()
print("Missing values in the DataFrame:")
print(missing_values)

Inspecting the Missing values Stop can be null as patient can have a disease diagnosed for life time cure example life style diseases like Hypertention

In [88]:
print("\nData types of each column:")
df_conditions.dtypes
## all the columns are of object type. we can cast it to desired type for e.g. convert date to date


Data types of each column:


START          object
STOP           object
PATIENT        object
ENCOUNTER      object
CODE            int64
DESCRIPTION    object
dtype: object

In [90]:
df_conditions["DESCRIPTION"].unique()
## few desc does not seems to be like disease name so removing it
## part-time employment (finding),
#Received higher education (finding)',
#      'Reports of violence in the environment (finding)',
#Has a criminal record (finding),
#Received certificate of   high school equivalency (finding)
#Full-time employment (finding)
list_invalid_values = ["Received certificate of high school equivalency (finding)","Full-time employment (finding)",
                      "Part-time employment (finding)","Part-time employment (finding)","Received higher education (finding)",
                      "Reports of violence in the environment (finding)","Received higher education (finding)"
                      ]
df_conditions = delete_from_data(df_conditions,"DESCRIPTION",list_invalid_values)
df_conditions["DESCRIPTION"].unique() ## removed all above

array(['Acute bronchitis (disorder)', 'Laceration of forearm',
       'Viral sinusitis (disorder)', 'Fracture of ankle',
       'Sprain of ankle', 'Risk activity involvement (finding)',
       'Body mass index 30+ - obesity (finding)', 'Hypertension',
       'Housing unsatisfactory (finding)',
       'Acute viral pharyngitis (disorder)', 'Stress (finding)',
       'Not in labor force (finding)', 'Limited social contact (finding)',
       'Refugee (person)', 'Social isolation (finding)',
       'Severe anxiety (panic) (finding', 'Sinusitis (disorder)',
       'Chronic sinusitis (disorder)', 'Normal pregnancy',
       'Laceration of thigh', 'Dyspnea (finding)', 'Wheezing (finding)',
       'Suspected COVID-19', 'COVID-19',
       'Victim of intimate partner abuse (finding)', 'Seizure disorder',
       'History of single seizure (situation)',
       'Served in armed forces (finding)', 'First degree burn',
       'Diabetes', 'Hypertriglyceridemia (disorder)',
       'Escherichia coli urina

In [92]:
df_conditions = clean_transform_condition_data(df_conditions)
df_conditions.head()

number of duplicates on encounter--> 0


,START_DIAG,STOP_DIAG,PATIENT_ID,ENCOUNTER,CODE,DESCRIPTION
0,2013-06-24,2013-07-02,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,0b2794bd-ec2b-d34f-0610-2523b3b7fcf0,10509002,Acute bronchitis (disorder)
1,2016-02-27,2016-03-14,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,a6d818dd-0983-fd1c-eefa-3d2295532c45,283371005,Laceration of forearm
2,2016-08-11,2016-08-22,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,36d2e781-4655-0a11-1f70-c69856e02019,444814009,Viral sinusitis (disorder)
3,2016-11-27,2016-12-17,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,c8eaaf41-958b-31ab-7de5-568cee8751f3,444814009,Viral sinusitis (disorder)
4,2017-02-22,2017-06-02,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,6474f606-5a1b-48c0-bbbf-ad6dcbc24d4e,16114001,Fracture of ankle


Applied the transformations 
Renaming the key columns
Convert Start and stop to required date time column types
Rename the columns

In [94]:
df_conditions.to_csv("./output/conditions_cleaned.csv",index=False)

##Export the cleaned data to output folder

In [ ]:
###### WORKING ON medication DATA 

In [96]:
print("\nData types of each column:")
df_medications.dtypes


Data types of each column:


START                 object
STOP                  object
PATIENT               object
PAYER                 object
ENCOUNTER             object
CODE                   int64
DESCRIPTION           object
BASE_COST            float64
PAYER_COVERAGE       float64
DISPENSES              int64
TOTALCOST            float64
REASONCODE           float64
REASONDESCRIPTION     object
dtype: object

In [98]:
missing_values = df_medications.isnull().sum()
print("Missing values in the DataFrame:")
print(missing_values)

Missing values in the DataFrame:
START                    0
STOP                  2713
PATIENT                  0
PAYER                    0
ENCOUNTER                0
CODE                     0
DESCRIPTION              0
BASE_COST                0
PAYER_COVERAGE           0
DISPENSES                0
TOTALCOST                0
REASONCODE           15471
REASONDESCRIPTION    15471
dtype: int64


No data in Stop column indicates the medications can be life time as an example Hypertention. ReasonCode and Reason description are not relevant fields at this atge of analysis

In [100]:
df_medications["DESCRIPTION"].unique()

array(['cetirizine hydrochloride 5 MG Oral Tablet',
       'NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector',
       'Acetaminophen 21.7 MG/ML / Dextromethorphan Hydrobromide 1 MG/ML / doxylamine succinate 0.417 MG/ML Oral Solution',
       'Ibuprofen 100 MG Oral Tablet',
       'Acetaminophen 160 MG Chewable Tablet',
       'Naproxen sodium 220 MG Oral Tablet',
       '1 ML medroxyPROGESTERone acetate 150 MG/ML Injection',
       'Acetaminophen 325 MG Oral Tablet',
       'Hydrochlorothiazide 25 MG Oral Tablet',
       'amLODIPine 2.5 MG Oral Tablet', 'lisinopril 10 MG Oral Tablet',
       'Amoxicillin 250 MG / Clavulanate 125 MG Oral Tablet',
       'Ibuprofen 200 MG Oral Tablet',
       'diphenhydrAMINE Hydrochloride 25 MG Oral Tablet',
       'Nitrofurantoin 5 MG/ML Oral Suspension',
       'Phenazopyridine hydrochloride 100 MG Oral Tablet',
       'Acetaminophen 325 MG Oral Tablet [Tylenol]',
       'NITROFURANTOIN  MACROCRYSTALS 50 MG Oral Capsule',
       'Penicillin V Potass

#Medication description data is clean and no Issues identified

In [102]:
df_medications = clean_transform_medication_data(df_medications)
df_medications.head()

,START_MED,STOP_MED,PATIENT_ID,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION,Year,Month
0,2020-02-17 10:40:32+00:00,2020-02-17 10:40:32+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,01efcc52-15d6-51e9-faa2-bee069fcbe44,1014676,cetirizine hydrochloride 5 MG Oral Tablet,21.45,0.0,21,450.45,NaN,NaN,2020,2
1,2020-02-17 10:40:32+00:00,2020-02-17 10:40:32+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,01efcc52-15d6-51e9-faa2-bee069fcbe44,1870230,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,231.58,0.0,21,4863.18,NaN,NaN,2020,2
2,2013-06-24 06:39:19+00:00,2013-07-02 06:39:19+00:00,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,0b2794bd-ec2b-d34f-0610-2523b3b7fcf0,1043400,Acetaminophen 21.7 MG/ML / Dextromethorphan Hy...,7.59,0.0,1,7.59,10509002.0,Acute bronchitis (disorder),2013,6
3,2016-02-27 08:52:29+00:00,2016-03-14 08:52:29+00:00,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,6e2f1a2d-27bd-3701-8d08-dae202c58632,a6d818dd-0983-fd1c-eefa-3d2295532c45,198405,Ibuprofen 100 MG Oral Tablet,12.48,0.0,1,12.48,NaN,NaN,2016,2
4,2017-02-22 09:25:31+00:00,2017-06-02 09:25:31+00:00,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,047f6ec3-6215-35eb-9608-f9dda363a44c,6474f606-5a1b-48c0-bbbf-ad6dcbc24d4e,313820,Acetaminophen 160 MG Chewable Tablet,4.77,0.0,3,14.31,NaN,NaN,2017,2


Below transformation have been applied on Medications data
1) rename the key ID to patient ID
2) Convert starta and stop to date time
3) retrive only relevent columns and renamed the columns

In [124]:
df_medications.to_csv("./output/medications_cleaned.csv",index=False)

#### Clean and transform Procedures data

In [104]:
print("\nData types of each column:")
df_procedures.dtypes


Data types of each column:


START                 object
STOP                  object
PATIENT               object
ENCOUNTER             object
CODE                   int64
DESCRIPTION           object
BASE_COST            float64
REASONCODE           float64
REASONDESCRIPTION     object
dtype: object

In [106]:
missing_values = df_procedures.isnull().sum()
print("Missing values in the DataFrame:")
print(missing_values)

Missing values in the DataFrame:
START                    0
STOP                     0
PATIENT                  0
ENCOUNTER                0
CODE                     0
DESCRIPTION              0
BASE_COST                0
REASONCODE           63226
REASONDESCRIPTION    63226
dtype: int64


In [108]:
df_procedures = clean_transform_procedure_data(df_procedures)
df_procedures

,START_PROC,STOP_PROC,PATIENT_ID,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION,Year,Month
0,2019-02-17 05:07:38+00:00,2019-02-17 05:22:38+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,748f8357-6cc7-551d-f31a-32fa2cf84126,430193006,Medication Reconciliation (procedure),608.11,NaN,NaN,2019,2
1,2019-05-26 05:07:38+00:00,2019-05-26 05:22:38+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,0bee1ce6-3e2c-5506-f71c-a7ba8f64a3d3,430193006,Medication Reconciliation (procedure),460.99,NaN,NaN,2019,5
2,2019-07-28 05:07:38+00:00,2019-07-28 05:22:38+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,6e93bcf9-45a4-8528-0120-1c1eaa930faf,430193006,Medication Reconciliation (procedure),492.11,NaN,NaN,2019,7
3,2020-02-17 10:07:38+00:00,2020-02-17 10:40:32+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,01efcc52-15d6-51e9-faa2-bee069fcbe44,395142003,Allergy screening test,516.65,NaN,NaN,2020,2
4,2020-04-26 05:07:38+00:00,2020-04-26 05:22:38+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,1a7debfc-9582-7f23-a109-4f154a182ee2,430193006,Medication Reconciliation (procedure),783.20,NaN,NaN,2020,4
...,...,...,...,...,...,...,...,...,...,...,...
83818,2021-01-15 05:54:11+00:00,2021-01-15 06:35:16+00:00,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,1516d2e6-4846-5f1e-fe27-c1ebb9a39f72,866148006,Screening for domestic abuse (procedure),516.65,NaN,NaN,2021,1
83819,2021-01-15 06:35:16+00:00,2021-01-15 06:50:05+00:00,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,1516d2e6-4846-5f1e-fe27-c1ebb9a39f72,171207006,Depression screening (procedure),516.65,NaN,NaN,2021,1
83820,2021-01-15 06:50:05+00:00,2021-01-15 07:10:18+00:00,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,1516d2e6-4846-5f1e-fe27-c1ebb9a39f72,454711000124102,Depression screening using Patient Health Ques...,516.65,NaN,NaN,2021,1
83821,2021-01-15 07:10:18+00:00,2021-01-15 07:21:00+00:00,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,1516d2e6-4846-5f1e-fe27-c1ebb9a39f72,428211000124100,Assessment of substance use (procedure),516.65,NaN,NaN,2021,1


Below transformation have been applied on Procedures data

1)rename the key ID to patient ID
2)Convert starta and stop to date time
3)retrive only relevent columns and renamed the columns

In [110]:
df_procedures.to_csv("./output/procedures_cleaned.csv",index=False)

Working on Encounters data and applying the silimar steps
1) Inspecting the value counts and checking for missing values
2) type casting to string from object type
3) converting starta nd stop to date time
4) output the cleaned csv to output folder

In [111]:
print("\nData types of each column:")
df_encounters.dtypes


Data types of each column:


Id                      object
START                   object
STOP                    object
PATIENT                 object
ORGANIZATION            object
PROVIDER                object
PAYER                   object
ENCOUNTERCLASS          object
CODE                     int64
DESCRIPTION             object
BASE_ENCOUNTER_COST    float64
TOTAL_CLAIM_COST       float64
PAYER_COVERAGE         float64
REASONCODE             float64
REASONDESCRIPTION       object
dtype: object

In [114]:
missing_values = df_encounters.isnull().sum()
print("Missing values in the DataFrame:")
print(missing_values)

Missing values in the DataFrame:
Id                         0
START                      0
STOP                       0
PATIENT                    0
ORGANIZATION               0
PROVIDER                   0
PAYER                      0
ENCOUNTERCLASS             0
CODE                       0
DESCRIPTION                0
BASE_ENCOUNTER_COST        0
TOTAL_CLAIM_COST           0
PAYER_COVERAGE             0
REASONCODE             45502
REASONDESCRIPTION      45502
dtype: int64


In [ ]:
missing_values = df_organizations.isnull().sum()
print("Missing values in the DataFrame:")
print(missing_values)

In [116]:
df_encounters = clean_transform_encounters_data(df_encounters)
df_encounters

,Id,START_ENC,STOP_ENC,PATIENT_ID,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION,LENGTH_OF_STAY,Year,Month
0,748f8357-6cc7-551d-f31a-32fa2cf84126,2019-02-17 05:07:38+00:00,2019-02-17 05:22:38+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,877.79,833.90,NaN,NaN,0.01,2019,2
1,5a4735ae-423f-6563-28ab-b3d11b49b2d4,2019-03-24 05:07:38+00:00,2019-03-24 05:22:38+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,269.68,256.20,NaN,NaN,0.01,2019,3
2,0bee1ce6-3e2c-5506-f71c-a7ba8f64a3d3,2019-05-26 05:07:38+00:00,2019-05-26 05:22:38+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,1292.75,1228.11,NaN,NaN,0.01,2019,5
3,6e93bcf9-45a4-8528-0120-1c1eaa930faf,2019-07-28 05:07:38+00:00,2019-07-28 05:22:38+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,1323.87,1257.68,NaN,NaN,0.01,2019,7
4,8b6787c3-4316-a0cb-899d-4746525c319f,2019-10-27 05:07:38+00:00,2019-10-27 05:22:38+00:00,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,831.76,790.17,NaN,NaN,0.01,2019,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61454,230e2215-38ab-9371-842d-a44d27ae4090,2020-12-18 05:11:58+00:00,2020-12-18 05:26:58+00:00,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,4861d01f-019c-3dac-a153-8334e50919f9,b0c5743a-4f9a-33b3-bab4-6a474fcf9ffd,d47b3510-2895-3b70-9897-342d681c769d,ambulatory,390906007,Follow-up encounter,77.49,77.49,0.00,55822004.0,Hyperlipidemia,0.01,2020,12
61455,db101ad8-66e2-9feb-e0cf-b2618f873c3a,2020-12-28 02:11:58+00:00,2020-12-28 02:26:58+00:00,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,4861d01f-019c-3dac-a153-8334e50919f9,b0c5743a-4f9a-33b3-bab4-6a474fcf9ffd,d47b3510-2895-3b70-9897-342d681c769d,ambulatory,185345009,Encounter for symptom,77.49,77.49,0.00,38822007.0,Cystitis,0.01,2020,12
61456,1516d2e6-4846-5f1e-fe27-c1ebb9a39f72,2021-01-15 05:11:58+00:00,2021-01-15 05:26:58+00:00,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,cd300275-3f4d-3adb-98aa-4e5bba25be24,7cd691a9-0e34-351a-918d-af3f6d3c0279,d47b3510-2895-3b70-9897-342d681c769d,wellness,162673000,General examination of patient (procedure),129.16,786.33,0.00,NaN,NaN,0.01,2021,1
61457,b2a4d90b-a2f5-1c88-0fb6-ba49b1487d37,2021-08-13 05:11:58+00:00,2021-08-13 05:26:58+00:00,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,4861d01f-019c-3dac-a153-8334e50919f9,b0c5743a-4f9a-33b3-bab4-6a474fcf9ffd,d47b3510-2895-3b70-9897-342d681c769d,outpatient,33879002,Administration of vaccine to produce active im...,129.16,269.68,0.00,NaN,NaN,0.01,2021,8


In [118]:
df_encounters.to_csv("./output/encounters_cleaned.csv",index=False)

#read Hospital data expecting this data can be used for analysis purpose

In [ ]:
df_organizationdata = df_organizations[['Id','NAME','ADDRESS','CITY','STATE','ZIP']]

string_cols = ['Id','NAME','ADDRESS','CITY','STATE']
for i in string_cols:
    df_organizationdata[i] = df_organizationdata[i].astype('string')

df_organizationdata['ZIP'] = df_organizationdata['ZIP'].astype('float64')



In [ ]:
""" Transformations"""
df_daignosisdatamerge = pd.merge(df_patientsdata, df_conditionsdata, on='PatientId', how='left')

df_diagnosisdata.drop(df_diagnosisdata.index, inplace=True)
# Select specific columns
df_diagnosisdata = df_daignosisdatamerge[['BIRTHDATE','START', 'STOP', 'PatientId', 'ENCOUNTER', 'DiagnosisCode',
       'DESCRIPTION']]


In [ ]:
df_diagnosisdata.columns


In [ ]:
df_daignosisdata['AGE_AT_DIAGNOSIS'] = ( df_daignosisdata['START']-df_daignosisdata['BIRTHDATE'] ).dt.days // 365

In [ ]:
pip install matplotlib

In [ ]:

import matplotlib.pyplot as plt

df_daignosisdatahist = df_daignosisdata[['AGE_AT_DIAGNOSIS' ,'PatientId']]
# Create histogram
plt.hist(df_daignosisdatahist, bins=10, edgecolor='black')

# Add labels and title
plt.xlabel(df_daignosisdatahist['AGE_AT_DIAGNOSIS'])
"""plt.ylabel(df_daignosisdatahist['PatientId'].value_counts())"""
plt.title('Histogram Example')

# Show the plot
plt.show()


In [ ]:
df_daignosisdatahist['PatientId'].value_counts()

In [ ]:
"""Define Age Groups"""


# Define bins and labels
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, float('inf')]
labels = ['0-5', '5-10', '10-15', '15-20', '20-25', '25-30', '30-35', '35-40', '40-45',
          '45-50', '50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90+']

# Categorize the ages
df_daignosisdata['Age Group'] = pd.cut(df_daignosisdata['AGE_AT_DIAGNOSIS'], bins=bins, labels=labels, right=False)




In [ ]:
df_daignosisdata.head()

In [ ]:
""" Length of Stay"""
df_encounterdata.head()

In [ ]:
""" Transformations"""
df_encounterdatamerge = pd.merge(df_patientsdata, df_encounterdata, on='PatientId', how='left')


# Select specific columns
df_hospitalvisitdata = df_encounterdatamerge[['BIRTHDATE','Id', 'START', 'STOP', 'PatientId', 'ORGANIZATION', 'ENCOUNTERCLASS',
       'CODE', 'DESCRIPTION']]
df_hospitalvisitdata.rename(columns={'Id': 'EncounterID'}, inplace=True)


In [ ]:
df_hospitalvisitdata.dtypes

In [ ]:
df_hospitalvisitdata['START'] = df_hospitalvisitdata['START'].dt.tz_localize(None)

df_hospitalvisitdata['AGE_AT_HospitalVisit'] = ( df_hospitalvisitdata['START']-df_hospitalvisitdata['BIRTHDATE'] ).dt.days // 365

In [ ]:
df_hospitalvisitdata['STOP'] = df_hospitalvisitdata['STOP'].dt.tz_localize(None)
df_hospitalvisitdata['length of Stay'] = (df_hospitalvisitdata['STOP'] - df_hospitalvisitdata['START']).dt.days


In [ ]:
df_encounterdata.head()

In [ ]:
/** NUmber of visits per patient
df_encounter